Prompt Engineering 

In [ ]:
# Naive Prompt

from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages= [ 
        {"role":"system", "content":"You are an AI Assistant"},
        {"role":"user", "content":"Can I have 10 product names for a pair of shoes that can fit any foot size?"}
    ],
    temperature=1.3,
)

print(response.choices[0].message.content)

Sure! Here are 10 product name ideas for shoes that can fit any foot size:

1. **FlexiFit Walkers**
2. **InfiniteStep Shoes**
3. **AdaptStride Sneakers**
4. **Universal Comfort Kicks**
5. **OneSize Wonder Wear**
6. **AnyFoot Adventure Boots**
7. **OmniFit Footwear**
8. **VersatileVue Shoes**
9. **ElasticEase Sneakers**
10. **Limitless Sole Sneakers**

Feel free to mix and match or modify them to best suit your brand!


In [ ]:
# Naive Prompt

from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages= [ 
        {"role":"system", "content":"You are an AI Assistant"},
        {"role":"user", "content":"Can I have 10 product names for a pair of shoes that can fit any foot size?"}
    ],
    temperature=1.3,
    
)

print(response.choices[0].message.content)

Sure! Here are 10 product name ideas for your versatile shoes that can fit any foot size:

1. **FlexiFit Slip-Ons**
2. **OneSize Wonder Walkers**
3. **AdaptEase Sneakers**
4. **Unisole Comfort Steps**
5. **SizeShift Footwear**
6. **Boundless Soles**
7. **Fit4Everyone Kicks**
8. **CushionFlex All-Sizers**
9. **Versatread Shoes**
10. **CustomFit Foot Comps**

Feel free to mix and match or modify them to suit your brand's image!


In [3]:
# Structured Prompt

prompt = """
## Role
You are a creative product naming assistant.

## Task
Generate 10 unique and catchy product names for a pair of shoes that can fit any foot size.

## Output
- Output must be in valid JSON.
- Return only an array of product name strings (no explanations, no numbering).
- Names should be short, memorable, and marketable.
- Avoid duplicates.

## Example
[
  "Name1",
  "Name2"
]
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an AI assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.8,
)

print(response.choices[0].message.content)


[
  "FlexFit",
  "SizeEase",
  "AdaptStep",
  "FitAll",
  "ComfyFlex",
  "OneSizeRun",
  "Universal Sole",
  "Mallea",
  "Shoes2You",
  "FitFolio"
]


CHAIN OF THOUGHT

In [ ]:
from openai import OpenAI

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # load variables from .env

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_completion(prompt, system):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


system = "Solve the following problem and return the answer in the format A: <answer>"
# Standard prompting 
question = """Q: How many Rs are there in RASPBERRY?"""

standard_prompt = f"""Q: How many Es are there in ELEPHANT?
A: 2
Q: How many Ps are there in PINEAPPLE?
A: 3
Q: How many Os are there in CHOCOLATE?
A: 2
{question}"""

print("Standard Prompt Result:")
print(get_completion(standard_prompt, system))

# Chain of Thought prompting
cot_prompt = f"""Q: How many Es are there in ELEPHANT?
Let's think step by step:
Spell out ELEPHANT and count the Es:
E-L-E-P-H-A-N-T
There's one E at the beginning and one in the middle. 1 + 1 = 2 Es in total.
A: 2

{question}. 
Let's think step by step:"""

print("\nChain of Thought Prompt Result:")
print(get_completion(cot_prompt, system))

Standard Prompt Result:
A: 2

Chain of Thought Prompt Result:
Spell out RASPBERRY and count the Rs:  
R-A-S-P-B-E-R-R-Y  
There's one R at the beginning and two in the middle. 1 + 2 = 3 Rs in total.  
A: 3


SELF - CONSISTENCY

In [ ]:
import asyncio
from openai import AsyncOpenAI
import nest_asyncio
from collections import Counter
import json
import os
import re

# Initialize the AsyncOpenAI client
client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
 
nest_asyncio.apply() # to run in a jupyter notebook async

async def generate_reasoning_path(question, temperature=0.7):
    """Generates a reasoning path using the OpenAI API asynchronously."""
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            # {"role": "system", "content": """You are a helpful assistant that provides step-by-step reasoning for questions. Provide a step-by-step reasoning and final answer as keys in JSON step_by_step_reasoning, and final_answer."""},
            {
        "role": "system",
        "content": """
        You are a helpful assistant that provides step-by-step reasoning for questions.
        Provide a step-by-step reasoning and final answer as keys in JSON:
        - step_by_step_reasoning
        - final_answer
        """
    },
            {"role": "user", "content": f"{question}"}
        ],
        temperature=temperature,
        max_tokens=1000
    )
    return response.choices[0].message.content

def extract_answer(reasoning_path: str) -> str:
    """Extracts the final answer from a reasoning path."""
    try:
        # Remove markdown fences if present
        cleaned = re.sub(r"```(?:json)?|```", "", reasoning_path).strip()
        parsed_json = json.loads(cleaned)
        return parsed_json.get("final_answer", "").replace("$", "")
    except json.JSONDecodeError:
        return ""

async def self_consistency(question, num_samples=5):
    """Implements the self-consistency method using the OpenAI API asynchronously."""
    tasks = [generate_reasoning_path(question) for _ in range(num_samples)]
    sampled_paths = await asyncio.gather(*tasks)
    answers = [extract_answer(path) for path in sampled_paths]
    most_common_answer = Counter(answers).most_common(1)[0][0]
    return most_common_answer, answers, sampled_paths

# Example usage
async def main():
    question = """Q: If there are 3 cars in the parking
    lot and 2 more cars arrive, how many
    cars are in the parking lot?
    A: {"step_by_step_reasoning": "There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.", "final_answer": "5"}

    Q: A factory produces widgets and gadgets. Each widget requires 3 units of material A and 2 units of material B. Each gadget requires 2 units of material A and 4 units of material B. The factory has 180 units of material A and 160 units of material B available. If the profit on each widget is $12 and on each gadget is $15, what is the maximum profit the factory can make?
    A:"""
    final_answer, answers, reasoning_paths = await self_consistency(question, num_samples=5)

    print(f"The most consistent answer is: {final_answer}")
    print("List of answers:")
    for i, answer in enumerate(answers, 1):
        print(f"Answer {i}: {answer}")
    print("\nReasoning paths:")
    for i, path in enumerate(reasoning_paths, 1):
        print(f"\nPath {i}:")
        print(path)

# Run the async main function
asyncio.run(main())

The most consistent answer is: 825
List of answers:
Answer 1: 660
Answer 2: 720
Answer 3: 825
Answer 4: 825
Answer 5: 960

Reasoning paths:

Path 1:
{"step_by_step_reasoning": "To solve this problem, we can use a method of linear programming to maximize the profit based on the material constraints. Let's define w as the number of widgets and g as the number of gadgets produced. The constraints based on material A and B are: 3w + 2g <= 180 (for material A) and 2w + 4g <= 160 (for material B). The objective function to maximize is the profit: P = 12w + 15g. We also have non-negativity constraints: w >= 0, g >= 0. We solve the system of inequalities to find feasible solutions. Using substitution or graphical methods, we find: At w = 40, g = 0, material A uses 120 units and material B uses 80 units, which satisfies both constraints. At w = 0, g = 40, material A uses 80 units and material B uses 160 units, which also satisfies both constraints. At w = 20, g = 30, material A uses 180 units a

ReAct Prompting:

In [8]:
import getpass
from openai import OpenAI
import re
import httpx

In [12]:
# Ref: Simon Willison https://til.simonwillison.net/llms/python-react-pattern
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client.api_key = os.getenv("OPENAI_API_KEY")

class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(model="gpt-4o", messages=self.messages)
        # print(completion.usage)
        return completion.choices[0].message.content

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()


action_re = re.compile(r'^Action: (\w+): (.*)$')


def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",  # Tell API we want to run a query
        "list": "search",   # We want search results
        "srsearch": q,      # Our search term (from function input)
        "format": "json"    # Return result in JSON format
    }).json()["query"]["search"][0]["snippet"]

def calculate(what):
    return eval(what)


known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

In [13]:
query("What is the capital of england?")

Thought: I should look up England on Wikipedia to find out its capital.
Action: wikipedia: England
PAUSE
 -- running wikipedia England
Observation: <span class="searchmatch">England</span> is a country that is part of the United Kingdom. It is located on the island of Great Britain, of which it covers about 62%, and more than 100
Answer: The capital of England is London.


In [14]:
query("What is 2 + 2?")

Thought: I can calculate the sum of 2 and 2 using arithmetic.
Action: calculate: 2 + 2
PAUSE
 -- running calculate 2 + 2
Observation: 4
Answer: 2 + 2 equals 4.


Personas of Thought

In [15]:
# Asking the LLM to generate a crowd of personas to answer a question before aggregating their feedback.
from openai import OpenAI
import os 
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
question = "I'm launching a new app that delivers personalized workout routines. What do you think of the name 'FitForYou'?"
number = 10

def call_openai(messages):
    # client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

system_prompt = "You are a helpful and terse assistant."
question_prompt = "I want a paragraph response to the following question: {question}"

naive_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question_prompt.format(question=question)}
]

naive_response = call_openai(naive_messages)

print(naive_response)


The name 'FitForYou' is a strong choice for your app, as it clearly conveys a focus on personalized fitness solutions tailored to individual needs. Its straightforward and catchy nature makes it memorable, which can enhance brand recognition. Additionally, the name implies a sense of inclusivity and adaptability, appealing to a wide range of users looking for workout routines that suit their unique goals and lifestyles. Overall, 'FitForYou' effectively communicates the app's purpose and value proposition.


In [16]:
experts_prompt = """I want a paragraph response to the following question: {question}

First, name {number} world-class experts (past or present) who would be great at answering this?

Then for each expert, please answer the question critically from their perspective given their background and experience.

Finally, combine the responses into a single response as if these experts had collaborated in writing a joint anonymous answer.

## Expert names:
Expert name (relevant experience)
Expert name (relevant experience)
Expert name (relevant experience)
...

### Expert responses:
Expert name: <response>
Expert name: <response>
Expert name: <response>
...
        
### Final response:
<joint response>
"""

experts_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": experts_prompt.format(question=question, number=number)}
]

experts_response = call_openai(experts_messages)

print(experts_response)

### Expert names:
1. Dr. Michael Joyner (Exercise Physiologist)
2. Jillian Michaels (Fitness Expert and Personal Trainer)
3. Dr. Greg Whyte (Sports Scientist and Olympian)
4. Tony Horton (Fitness Trainer and Creator of P90X)
5. Dr. John Ratey (Neuroscientist and Author)
6. Kayla Itsines (Fitness Influencer and App Creator)
7. Dr. Tim Noakes (Sports Scientist and Author)
8. Richard Simmons (Fitness Advocate and Motivational Speaker)
9. Dr. Rachael O'Meara (Behavioral Scientist and Author)
10. Chris Powell (Transformation Specialist and Fitness Expert)

### Expert responses:
Dr. Michael Joyner: "The name 'FitForYou' effectively communicates personalization and engages users by suggesting a tailored approach to fitness, which is essential for adherence and success."

Jillian Michaels: "'FitForYou' is catchy and invites users to feel that the app caters specifically to their individual needs, which is crucial in a market saturated with generic fitness solutions."

Dr. Greg Whyte: "The name

In [17]:
personas_prompt = """I want a paragraph response to the following question: {question}

First, name {number} demographic personas who would be relevant for answering this?

Then for each persona, please answer the question critically from their perspective given their background and experience.

Finally, combine the responses into a single response as if these personas had collaborated in writing a joint anonymous answer.

### Persona names:
Persona name (relevant demographics)
Persona name (relevant demographics)
Persona name (relevant demographics)
...

### Persona responses:
Persona name: <response>
Persona name: <response>
Persona name: <response>
...

### Final response:
<joint response>
"""

personas_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": personas_prompt.format(question=question, number=number)}
]

personas_response = call_openai(personas_messages)

print(personas_response)

### Persona names:
1. Sarah (25, fitness enthusiast, social media influencer)
2. Mark (32, busy professional, occasional gym-goer)
3. Lisa (40, mother of two, looking to get back in shape)
4. James (28, college student, interested in strength training)
5. Emily (35, health coach, focused on holistic wellness)
6. Tom (45, tech-savvy, values innovation in fitness)
7. Rachel (22, first-time gym user, seeking guidance)
8. David (30, athlete, interested in performance optimization)
9. Karen (50, retired, interested in low-impact exercises)
10. Alex (27, traveler, prefers flexible workout options)

### Persona responses:
**Sarah:** The name 'FitForYou' has a personal touch, which is great for attracting users who want tailored experiences. However, it may lack uniqueness in a crowded market.

**Mark:** I think 'FitForYou' conveys a sense of personalization, which is appealing. But I wonder if it will stand out among other fitness apps with similar names.

**Lisa:** As someone looking to rega

In [18]:
def extract_final_response(response):
    return response.split("### Final response:")[1].strip()

experts_final_response = extract_final_response(experts_response)
personas_final_response = extract_final_response(personas_response)

import textwrap

def pretty_print(label, text, width=80):
    wrapped = textwrap.fill(text, width=width)
    print(f"{label}:\n{wrapped}\n{'-'*100}")

pretty_print("Naive", naive_response)
pretty_print("Experts", experts_final_response)
pretty_print("Personas", personas_final_response)

Naive:
The name 'FitForYou' is a strong choice for your app, as it clearly conveys a
focus on personalized fitness solutions tailored to individual needs. Its
straightforward and catchy nature makes it memorable, which can enhance brand
recognition. Additionally, the name implies a sense of inclusivity and
adaptability, appealing to a wide range of users looking for workout routines
that suit their unique goals and lifestyles. Overall, 'FitForYou' effectively
communicates the app's purpose and value proposition.
----------------------------------------------------------------------------------------------------
Experts:
The name 'FitForYou' effectively captures the essence of personalized fitness,
emphasizing individual journeys and tailored workout routines. Experts agree
that this name is catchy and relatable, making it appealing in a competitive
market. It suggests a commitment to personalization, which is crucial for user
engagement and motivation. However, the success of the app w

In [19]:
judge_prompt = """I want you to compare two responses to the following question: {question}

Here are the two responses:

Response 1:
{response1}

Response 2:
{response2}

Please analyze both responses and choose which one is better. Consider factors like:
- Uniqueness of perspectives
- Nonobvious insights
- Sounds human not like an AI

## Your analysis:
<analysis>

## Your choice:
The better response is: Response <1 or 2>
"""

def judge_responses(response1, response2):
    judge_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": judge_prompt.format(
            question="How can I be more productive?",
            response1=response1,
            response2=response2
        )}
    ]

    judge_response = call_openai(judge_messages)

    choice = judge_response.split("The better response is: Response ")[1].strip()
    return {"choice": int(choice), "reason": judge_response}


naive_vs_experts = judge_responses(naive_response, experts_final_response)
print("# Naive vs experts:", naive_vs_experts["choice"])

naive_vs_personas = judge_responses(naive_response, personas_final_response)
print("# Naive vs personas:", naive_vs_personas["choice"])

experts_vs_personas = judge_responses(experts_final_response, personas_final_response)
print("# Experts vs personas:", experts_vs_personas["choice"])

print("-"*100)
print(naive_vs_experts)
print("-"*100)
print(naive_vs_personas)
print("-"*100)
print(experts_vs_personas)

# Naive vs experts: 2
# Naive vs personas: 2
# Experts vs personas: 2
----------------------------------------------------------------------------------------------------
{'choice': 2, 'reason': "<analysis>\nBoth responses effectively discuss the name 'FitForYou' and its implications for a fitness app. However, they differ in depth and insight.\n\nResponse 1 provides a straightforward analysis that highlights the name's clarity, memorability, and inclusivity. While it covers fundamental aspects, it lacks unique perspectives and deeper insights into user engagement or market competition.\n\nResponse 2, on the other hand, offers a richer evaluation. It not only discusses the name's appeal and relevance but also incorporates expert opinions and emphasizes the importance of delivering on the promise of personalization. This response addresses user engagement and motivation, which adds layers to the analysis. Furthermore, it acknowledges market competition and the broader demand for persona